In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np 
import pandas as pd 
import re
import matplotlib.pyplot as plt
import seaborn as sns
import collections
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, classification_report

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current sessionimport warnings
warnings.filterwarnings('ignore')
import numpy as np 
import pandas as pd 
import re
import matplotlib.pyplot as plt
import seaborn as sns
import collections
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, classification_report

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_v1 = pd.read_json('../input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset.json', lines = True)
df_v2 = pd.read_json('../input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset_v2.json', lines = True)

In [ ]:
df_v1.head()

In [ ]:
df_v2.head()

In [ ]:
df = pd.concat([df_v1, df_v2], axis=0)

In [ ]:
df.shape


In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.drop('article_link', axis=1, inplace=True)

In [ ]:
df.columns = ['text', 'labels']


In [ ]:
df.duplicated().sum()


In [ ]:
df.drop_duplicates(keep='first', inplace=True)

In [ ]:
df.info()

In [ ]:
df['labels'].value_counts()

In [ ]:
sns.countplot(x= df['labels'])
plt.show()

In [ ]:
sns.histplot(df['labels'], kde=True)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(x='labels', data=df)

sns.stripplot(x='labels', data=df, color='black', jitter=True)  # Add strip or swarm plot on top

plt.show()


In [ ]:
regex = re.compile('[^a-z\s]')

df['text'] = df['text'].str.lower()
df['text'] = df['text'].apply(lambda x: regex.sub('', x))
df['text'] = df['text'].str.replace('rt|http', '', regex = True)
df['text'] = df['text'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 2]))

In [ ]:
df['text'] = df['text'].apply(word_tokenize)

stop_words = set(stopwords.words('english'))
df['text'] = df['text'].apply(lambda x: [word for word in x if word not in stop_words])

In [ ]:
stemmer = SnowballStemmer('english')
df['text'] = df['text'].apply(lambda x: [stemmer.stem(w) for w in x])
# Joining the words back into a single text
df['text'] = df['text'].apply(lambda x: ' '.join(x))

In [ ]:
all_words = ' '.join(word for word in df['text'])

In [ ]:
wordcloud = WordCloud(width = 800, height = 500, background_color = 'black', 
                min_font_size = 10).generate(all_words)

plt.figure(figsize = (10, 6), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis('off') 
plt.tight_layout(pad = 0) 
plt.show()

In [ ]:
X = np.array(df['text'])
y = np.array(df['labels'])

In [ ]:
cv = CountVectorizer()
X = cv.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state= 42)

In [ ]:
bnb = BernoulliNB()
bnb.fit(X_train, y_train)

In [ ]:
y_pred = bnb.predict(X_test)

In [ ]:
print("Classification Report: \n", classification_report(y_test, y_pred))
